# Sociation2Vec
Embedding-и слов аналогичные word2vec из базы ассоциаций [Sociation.org](http://sociation.org)

In [1]:
import os
from operator import itemgetter
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as progress
import gensim

In [2]:
source_dir = "./source_corpus"
data_dir = "/Users/ur001/Documents/Sociation/sociation2vec_model"
random_state = 777

%pylab inline
%config InlineBackend.figure_format = 'svg' 
%config InlineBackend.figure_format = 'retina'

# отключаем ворнинги
import warnings
warnings.simplefilter('ignore')

# дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 14,9

Populating the interactive namespace from numpy and matplotlib


### Лучший на данный момент вариант
 - corpus
 - normalize_matrix
 - ppmi (0.75, 5.5)
 - tfidf (по словам)
 - svd 790
 - normalize_matrix
 - normalize_l2 (при получении вектора слова)
 
```python
corpus, words_dict, assoc_dict = load_source_corpus()
corpus_csc = corpus_to_sparse(corpus)
corpus_csc = normalize_matrix(corpus_csc, with_mean=False)
corpus_ppmi = get_ppmi_weight(corpus_csc, cds=0.75, k=5.5)
tfidf, corpus_ppmi = get_tfidf(corpus_ppmi, smooth_idf=False)
svd, corpus_ppmi_svd = get_svd(corpus_ppmi, 790)
corpus_ppmi_svd = normalize_matrix(corpus_ppmi_svd)
similarity_index = create_similarity_index(corpus_ppmi_svd)
sociation2vec = Sociation2Vec(svd, corpus_ppmi_svd, similarity_index, words_dict, assoc_dict)
```

## Загрузка словарей и корпуса ассоциаций

In [3]:
%load_ext autoreload
%autoreload 2
from sociation2vec.utils import DictEncoder, save_word2vec, read_word2vec
from sociation2vec.model import Sociation2Vec
from sociation2vec import mathutils, model_builder, pandas_utils

In [4]:
corpus, words_dict, assoc_dict = model_builder.load_source_corpus(source_dir)
words_count = len(words_dict.decode)
assoc_count = len(assoc_dict.decode)
print("Слов: {}, Слов-ассоциаций: {}".format(words_count, assoc_count))

Loading source corpus...
Loading words index...
Loading assoc words index...
Words: 29903, Assoc words: 49424
Слов: 29903, Слов-ассоциаций: 49424


In [5]:
pandas_utils.get_associations('система', corpus=corpus, words_dict=words_dict, assoc_dict=assoc_dict).head()

,popularity
word,
порядок,6.54545
структура,5.33333
государство,4.81250
виндоус,4.00000
ниппель,3.00000


## Создание модели вручную

### Нормализация, PPMI, TF-IDF

In [6]:
corpus_csc = mathutils.corpus_to_sparse(corpus, words_count, assoc_count)
corpus_csc.shape, (words_count, assoc_count)

((29903, 49424), (29903, 49424))

In [7]:
corpus_csc = mathutils.normalize_matrix(corpus_csc, with_mean=False)

In [8]:
corpus_ppmi = mathutils.get_ppmi_weight(corpus_csc, cds=0.75, k=5)
corpus_ppmi.shape

(29903, 49424)

In [9]:
tfidf, corpus_ppmi = mathutils.get_tfidf(corpus_ppmi)
corpus_ppmi.shape

(29903, 49424)

In [10]:
pandas_utils.get_associations(
    'система', 
    corpus=mathutils.sparse_to_corpus(corpus_ppmi),
    words_dict=words_dict, 
    assoc_dict=assoc_dict
).head(10)

,popularity
word,
симплекс-метод,0.173802
энергоснабжение,0.173802
комплекс мер,0.173802
подсистема,0.158627
операционка,0.152325
функциональность,0.143095
хря,0.128959
метаистория,0.126786
компоненты,0.115653


### Факторизация PPMI-матрицы с помощью SVD

In [11]:
# Загрузка корпуса и SVD модели
corpus_ppmi_svd = model_builder.load_corpus('corpus_norm_ppmi_0.75_5.5_tfidf_svd_norm_790', data_dir=data_dir)
svd = model_builder.load_model('model_norm_ppmi_0.75_5.5_tfidf_svd_norm_790', data_dir=data_dir)

Loading corpus from /Users/ur001/Documents/Sociation/sociation2vec_model/corpus_norm_ppmi_0.75_5.5_tfidf_svd_norm_790.npy...
Loading model from /Users/ur001/Documents/Sociation/sociation2vec_model/model_norm_ppmi_0.75_5.5_tfidf_svd_norm_790.pkl...


**ИЛИ**

In [ ]:
# Создание и сохранение корпуса и SVD модели
svd, corpus_ppmi_svd = mathutils.get_svd(corpus_ppmi, 800)
corpus_ppmi_svd = mathutils.normalize_matrix(corpus_ppmi_svd)
model_builder.save_corpus(corpus_ppmi_svd, 'corpus_norm_ppmi_svd_norm_800', data_dir=data_dir)
model_builder.save_model(svd, 'svd_800_norm_ppmi_model', data_dir=data_dir)

In [ ]:
corpus_ppmi_svd.shape

### Загрузка или создание/сохранение индекса

In [13]:
similarity_index = model_builder.create_or_load_similarity_index(
    corpus_ppmi_svd, 
    'norm_ppmi_0.75_5.5_tfidf_svd_norm_790', 
    data_dir=data_dir
)

Loading similarity index from /Users/ur001/Documents/Sociation/sociation2vec_model/norm_ppmi_0.75_5.5_tfidf_svd_norm_790.index


## Смотрим что вышло

In [14]:
sociation2vec = Sociation2Vec(svd, corpus_ppmi_svd, similarity_index, words_dict, assoc_dict)

In [15]:
get_similar = pandas_utils.build_get_similar(sociation2vec)

In [16]:
get_similar('король,-мужчина,женщина')

,similarity
word,
королева,0.564993
корона,0.501877
царица,0.474081
монарх,0.441997
королевство,0.441149
царство,0.432549
монархия,0.431903
трон,0.422036
царь,0.417895


### Recommendations
http://blog.aylien.com/overview-word-embeddings-history-word2vec-cbow-glove/

- DON’T use shifted PPMI with SVD. (**враньё!**)
- DON’T use SVD “correctly”, i.e. without eigenvector weighting (performance drops 15 points compared to with eigenvalue weighting with p=0.5). (**не помогло**)
- DO use PPMI and SVD with short contexts (window size of 2).
- DO use many negative samples with SGNS.
- DO always use context distribution smoothing (raise unigram distribution to the power of α=0.75) for all methods.
- DO use SGNS as a baseline (robust, fast and cheap to train).
- DO try adding context vectors in SGNS and GloVe.

## Автоматизация создания модели

In [17]:
sociation2vec_builder = model_builder.ModelBuilder(source_path=source_dir, out_path=data_dir)
sociation2vec = sociation2vec_builder.get_model(
    ppmi_k=5.5, 
    svd_components=790, 
    tfidf_smooth_tf=False
)

Loading source corpus...
Loading words index...
Loading assoc words index...
Words: 29903, Assoc words: 49424
Normalizing source sorpus...
Applying PPMI cds=0.75, neg=5.5...
Applying TFIDF smooth_tf=False...
Loading corpus from /Users/ur001/Documents/Sociation/sociation2vec_model/corpus_norm_ppmi_0.75_5.5_tfidf_svd_norm_790.npy...
Loading model from /Users/ur001/Documents/Sociation/sociation2vec_model/model_norm_ppmi_0.75_5.5_tfidf_svd_norm_790.pkl...
Loading similarity index from /Users/ur001/Documents/Sociation/sociation2vec_model/norm_ppmi_0.75_5.5_tfidf_svd_norm_790.index


In [ ]:
# sociation2vec.save_word2vec('sociation2vec820.vec')

python evaluate.py ../sociation2vec820.vec 'russian'

    SimLex-999 score and coverage: 0.556 599
    WordSim overall score and coverage: 0.808 305
    WordSim Similarity score and coverage: 0.795 177
    WordSim Relatedness score and coverage: 0.807 220
    
python evaluate.py ../araneum_upos_skipgram_300_2_2018.vec 'russian'

    SimLex-999 score and coverage: 0.398 654
    WordSim overall score and coverage: 0.636 315
    WordSim Similarity score and coverage: 0.722 182
    WordSim Relatedness score and coverage: 0.546 225 

In [18]:
get_similar = pandas_utils.build_get_similar(sociation2vec)
compare = pandas_utils.build_compare(sociation2vec)

In [19]:
get_similar('король,-мужчина,женщина')

,similarity
word,
королева,0.564993
корона,0.501877
царица,0.474081
монарх,0.441997
королевство,0.441149
царство,0.432549
монархия,0.431903
трон,0.422036
царь,0.417895


In [20]:
get_similar('работа,-труд')

,similarity
word,
офис,0.422823
подчинённый,0.407866
менеджер,0.364413
сотрудник,0.353013
клерк,0.344958
офисный планктон,0.333883
будни,0.326154
контора,0.325322
должность,0.321485


In [21]:
get_similar('труд,работа')

,similarity
word,
работник,0.607796
трудолюбие,0.590474
трудяги,0.57155
трудоголик,0.554924
трудяга,0.546713
занятость,0.534091
усердие,0.519577
работяга,0.504307
трудоустройство,0.502201


In [22]:
compare('работа', 'труд', 8, similarity_degree=0.75, separate=True, min_score=0.45)

,работа,score1,общие,score_common,труд,score2
0,подчинённый,0.462564,работник,0.411929,трудяги,0.397131
1,офис,0.427838,трудолюбие,0.392955,рабочий класс,0.396830
2,сотрудник,0.390770,трудоголик,0.361773,трудящиеся,0.394108
3,будни,0.378630,трудяга,0.352422,усердие,0.361746
4,клерк,0.374348,занятость,0.336878,труженик,0.358488
5,должность,0.367100,трудоустройство,0.311097,пролетариат,0.348899
6,зарплата,0.336693,работоспособность,0.309025,старание,0.339852
7,карьера,0.330384,деятельность,0.300580,усилия,0.337452


In [23]:
compare('мужчина,мужик', 'женщина,девушка', 6, similarity_degree=0.44, separate=True, min_score=0.43)

,"мужчина,мужик",score1,общие,score_common,"женщина,девушка",score2
0,мужлан,0.550174,муж,0.429303,красавица,0.623013
1,мачо,0.549981,баба,0.411673,красотка,0.592361
2,мужественность,0.547201,жена,0.385147,дева,0.566830
3,брутальность,0.498491,,,женственность,0.523137
4,красавчик,0.497456,,,девица,0.518643
5,мужичок,0.488264,,,красота,0.516660


In [24]:
compare('самец', 'самка', 8, similarity_degree=0.62, separate=True, min_score=0.36)

,самец,score1,общие,score_common,самка,score2
0,альфа-самец,0.501640,оплодотворение,0.392204,окот,0.371033
1,мачо,0.453056,яйцеклетка,0.356770,роды,0.353193
2,бабник,0.404416,размножение,0.355228,беременность,0.352849
3,полигамия,0.401384,особь,0.326233,женщина,0.350184
4,жиголо,0.376624,осеменитель,0.307503,продолжение рода,0.347874
5,альфа,0.375033,кобель,0.297281,матка,0.342234
6,сердцеед,0.373353,осеменение,0.292061,эко,0.329424
7,мужлан,0.369311,львица,0.288839,детёныши,0.328449


In [25]:
compare('день,-день недели', 'ночь', 5, similarity_degree=0.63, min_score=0.39)

,"день,-день недели",score1,общие,score_common,ночь,score2
0,рань,0.323815,сумерки,0.469051,темнота,0.511563
1,восход,0.319845,вечер,0.450629,звёздная ночь,0.480745
2,заря,0.318036,рассвет,0.390697,сумрак,0.476020
3,солнце,0.309521,полночь,0.388152,полумрак,0.469653
4,,,полдень,0.377098,тьма,0.452538


In [26]:
compare('жизнь', 'смерть', 5, similarity_degree=0.6, min_score=0.35)

,жизнь,score1,общие,score_common,смерть,score2
0,настоящее,0.457429,вечность,0.382423,похороны,0.659941
1,существование,0.440883,страдание,0.308237,гроб,0.651559
2,реальность,0.438629,бессмертие,0.295687,могила,0.641188
3,смысл жизни,0.431155,бренность,0.287133,кладбище,0.634403
4,быстротечность,0.419550,реинкарнация,0.265099,покойник,0.603098


In [27]:
compare('мозг', 'сознание', similarity_degree=0.7, min_score=0.5)

,мозг,score1,общие,score_common,сознание,score2
0,черепная коробка,0.623955,разум,0.575432,бессознательность,0.600099
1,ганглий,0.608758,мышление,0.504414,осознанность,0.586550
2,неокортекс,0.584746,ум,0.503366,бытие,0.520027
3,гипоталамус,0.577190,рассудок,0.467008,сущность,0.508941
4,соображалка,0.569312,интеллект,0.445933,сознательность,0.495589
5,серое вещество,0.547399,мысль,0.439252,самосознание,0.492071
6,гиппокамп,0.533102,мысли,0.379851,осознание,0.481553
7,нервная система,0.529149,,,бессознательное,0.480710
8,синапс,0.528874,,,осмысление,0.440365
9,мозги,0.527388,,,просветление,0.432866


In [28]:
compare('любовь', 'секс', 6, similarity_degree=0.7, min_score=0.34)

,любовь,score1,общие,score_common,секс,score2
0,преданность,0.525199,страсть,0.499792,мастурбация,0.590357
1,верность,0.499681,удовольствие,0.445615,порно,0.584603
2,супруги,0.497304,близость,0.431357,половой акт,0.558624
3,доверие,0.481097,взаимность,0.414319,сладострастие,0.536469
4,искренность,0.434489,нежность,0.390235,эротика,0.520234
5,привязанность,0.434192,наслаждение,0.374653,сперма,0.518000


In [29]:
compare('лист', 'листья', 6, similarity_degree=0.7, min_score=0.3)

,лист,score1,общие,score_common,листья,score2
0,блокнот,0.497571,листва,0.651231,деревья,0.393826
1,тетрадь,0.484297,листок,0.601661,каштаны,0.373156
2,бумага,0.458839,клён,0.516046,осень,0.367731
3,записи,0.421504,листопад,0.439947,опавшая листва,0.353449
4,листочек,0.420209,дерево,0.431742,аллея,0.348635
5,чистовик,0.408549,ветка,0.407278,осинник,0.347191


In [30]:
compare('сталин', 'гитлер', 9, similarity_degree=0.6, min_score=0.45)

,сталин,score1,общие,score_common,гитлер,score2
0,совет министров,0.493942,тоталитаризм,0.482524,нацист,0.778091
1,ленин,0.479279,диктатор,0.456276,арийцы,0.723780
2,ссср,0.475097,репрессии,0.428068,рейх,0.673356
3,коммунизм,0.433725,узурпатор,0.423402,нацизм,0.672314
4,берия,0.428008,тиран,0.368281,фриц,0.670448
5,социализм,0.425485,большевизм,0.359827,сс,0.660512
6,тактик,0.398646,идеология,0.358307,третий рейх,0.658504
7,авторитарность,0.357649,тирания,0.337973,холокост,0.657885
8,вождь,0.347106,,,фашист,0.656994


In [31]:
get_similar('король,-брат,сестра', 1)

,similarity
word,
королева,0.620279


In [32]:
get_similar('король,-мальчик,девочка', count=1)

,similarity
word,
королева,0.543709


In [33]:
get_similar('ассоциация', 15)

,similarity
word,
синоним,0.648149
гипоним,0.647139
ассоциации,0.633215
синонимы,0.614983
аналогия,0.583233
схожесть,0.567518
общность,0.548526
подобие,0.545771
похожесть,0.541983


In [34]:
get_similar('жизнь', count=3)

,similarity
word,
существование,0.5739
бытие,0.54969
рождение,0.493273


In [35]:
get_similar('симбиоз,-паразит', 15)

,similarity
word,
коллаборация,0.535389
кооперация,0.488304
сотрудничество,0.416851
взаимовыгода,0.416784
союз,0.406464
объединение,0.399354
товарищество,0.397312
партнерство,0.395137
дружба,0.379424


In [36]:
get_similar('система,-операционная система')

,similarity
word,
структура,0.400392
упорядоченность,0.373541
закономерность,0.354664
системы,0.350485
порядок,0.34256
математика,0.319886
взаимосвязь,0.317276
сортировка,0.30686
статистика,0.303903


In [37]:
get_similar('ватник,быдло', count=2)

,similarity
word,
гопник,0.628263
гопота,0.558603


In [38]:
get_similar('ватник,-быдло', count=10)

,similarity
word,
телогрейка,0.461277
анорак,0.401182
пуховик,0.392203
куртка,0.373681
толстовка,0.369719
худи,0.356558
выворотка,0.346021
парка,0.343121
подкладка,0.33732


In [39]:
get_similar('бесконечность')

,similarity
word,
вселенная,0.693702
вечность,0.656467
безграничность,0.635307
континуум,0.617114
непостижимость,0.590898
пространство,0.555309
дальний космос,0.53323
промежуток,0.524914
чёрная дыра,0.518758


In [40]:
get_similar('вечность')

,similarity
word,
бесконечность,0.656467
время,0.592194
минута,0.549076
промежуток,0.534839
мгновение,0.5348
быстротечность,0.5342
бытие,0.532054
секунда,0.527471
длительность,0.512995


In [41]:
get_similar('плотность,объём', count=1)

,similarity
word,
масса,0.534643


In [42]:
get_similar('кошка,-женщина,мужчина', count=3)

,similarity
word,
кот,0.565539
котик,0.459855
котёнок,0.457098


In [43]:
get_similar('мама,-женщина,мужчина', count=2)

,similarity
word,
папа,0.569135
отец,0.564594


In [44]:
get_similar('папа,-мужчина,женщина', count=2)

,similarity
word,
мама,0.586447
мать,0.557549


In [45]:
get_similar('сын,-мужчина,женщина', count=1)

,similarity
word,
дочь,0.641909


In [46]:
get_similar('дочь,-женщина,мужчина', count=1)

,similarity
word,
сын,0.589326


In [47]:
get_similar('брат,-мальчик,девочка', count=1)

,similarity
word,
сестра,0.581946


In [48]:
get_similar('принц,-мальчик,девочка', count=1)

,similarity
word,
принцесса,0.567035


In [49]:
get_similar('жена,-девочка,мальчик', count=2)

,similarity
word,
муж,0.584988
супруг,0.500036


In [50]:
get_similar('ворон,-мужчина,женщина', count=1)

,similarity
word,
ворона,0.426823


In [51]:
get_similar('волк,-мужчина,женщина', count=2)

,similarity
word,
волки,0.442508
хищница,0.441653


In [52]:
get_similar('собака,-женщина,мужчина', count=1)

,similarity
word,
пёс,0.61831


In [53]:
get_similar('бог,-христианство,индуизм', count=3)

,similarity
word,
шива,0.725105
брахма,0.712545
многорукость,0.635756


In [54]:
get_similar('человек,-человечность')

,similarity
word,
существо,0.338447
особь,0.323041
обезьяна,0.309612
горилла,0.296778
дарвин,0.275694
животное,0.275616
примат,0.274331
млекопитающее,0.273347
бабуин,0.265634


In [55]:
get_similar('человек,человечность')

,similarity
word,
филантроп,0.561842
гуманизм,0.551552
гуманность,0.527699
хомо сапиенс,0.506428
разум,0.49996
индивид,0.497178
человеколюбие,0.494812
личность,0.492035
человечество,0.491332


In [56]:
get_similar('помидоры,-красный цвет,зелёный цвет', count=2)

,similarity
word,
огурец,0.598731
огурцы,0.569421


In [57]:
get_similar('огурцы,-зелёный цвет,красный цвет', count=2)

,similarity
word,
помидор,0.504272
помидоры,0.502661


In [58]:
get_similar('закат,-вечер,утро', count=2)

,similarity
word,
рассвет,0.718387
восход,0.670692


In [59]:
get_similar('свет,-день,ночь', count=2)

,similarity
word,
освещение,0.646371
темнота,0.639946


In [60]:
get_similar('похолодание,-осень,весна', count=2)

,similarity
word,
оттепель,0.744509
потепление,0.726601


In [61]:
get_similar('ключ,вода', count=1)

,similarity
word,
родник,0.635108


In [62]:
get_similar('ключ,болт', count=1)

,similarity
word,
гайка,0.774266


In [63]:
get_similar('ключ,криптография', count=1)

,similarity
word,
шифр,0.754577


In [64]:
get_similar('замок,ключ', count=3)

,similarity
word,
отмычка,0.695543
замочек,0.692001
ключик,0.658589


In [65]:
get_similar('замок,средневековье', count=2)

,similarity
word,
замки,0.657284
рыцари,0.61809


In [66]:
get_similar('папа,мама', count=4)

,similarity
word,
отец,0.765879
мать,0.726327
родитель,0.71556
родители,0.68037


In [67]:
get_similar('квадрат', count=7)

,similarity
word,
прямоугольник,0.805408
параллелепипед,0.764639
треугольник,0.760536
ромб,0.744727
четырёхугольник,0.742359
куб,0.735296
прямоугольность,0.73469


In [68]:
get_similar('роберт шекли', count=10)

,similarity
word,
роджер желязны,0.798741
фантаст,0.727146
вербер,0.707533
дуглас адамс,0.686447
станислав лем,0.673586
рэй брэдбери,0.646987
александр беляев,0.646181
айзек азимов,0.642642
голова профессора доуэля,0.58137


In [69]:
get_similar('тимур и его команда', count=10)

,similarity
word,
скауты,0.508559
пионеры,0.446565
мальчиш-кибальчиш,0.440232
тимур,0.42047
пионерский лагерь,0.411418
октябрята,0.405712
горнист,0.380901
общий сбор,0.375745
гайдар,0.372595


In [70]:
get_similar('инструмент', count=10)

,similarity
word,
молоток,0.649512
монтировка,0.630254
пассатижи,0.615171
инструменты,0.613172
киянка,0.60671
аппликатор,0.60239
долото,0.588334
гвоздодёр,0.573839
напильник,0.567663


In [71]:
get_similar('медвежуть', count=10)

,similarity
word,
берлога,0.392086
бурый медведь,0.388666
мультяшка,0.380729
норштейн,0.375027
топтыгин,0.368594
педобир,0.358309
гризли,0.344227
семь гномов,0.335821
топтыжка,0.324979


In [72]:
get_similar('доброта')

,similarity
word,
добро,0.77657
добродушие,0.721765
сердечность,0.715634
радушие,0.705052
великодушие,0.697467
добродетель,0.695517
милосердие,0.678244
сострадание,0.672974
доброжелательность,0.667854


In [73]:
get_similar('безвозмездность', count=4)

,similarity
word,
бескорыстие,0.777126
бескорыстность,0.713216
благотворительность,0.690965
альтруизм,0.640819


In [74]:
get_similar('странность', count=5)

,similarity
word,
своеобразие,0.560963
необычность,0.538117
неординарность,0.521447
уникальность,0.500191
нестандартность,0.492364


In [75]:
get_similar('женщина,девушка,-мужчина,-мужик', count=2)

,similarity
word,
красотка,0.443778
красавица,0.435511


In [76]:
get_similar('мужчина,мужик,-женщина,-девушка', count=3)

,similarity
word,
мужлан,0.370184
брутальность,0.341347
мачо,0.330942


In [77]:
get_similar('соль,нота', count=5)

,similarity
word,
диез,0.799394
бемоль,0.791552
ля,0.788204
фа,0.786435
ре,0.784228


In [78]:
get_similar('соль,-нота')

,similarity
word,
соли,0.422692
перец,0.418025
уксус,0.382945
хлорид натрия,0.376353
горчица,0.375457
сода,0.358736
солонка,0.357009
приправа,0.356154
рассол,0.34116


In [79]:
get_similar('рикки-тикки-тави', count=3)

,similarity
word,
мангуст,0.711756
маугли,0.561029
киплинг,0.556536


In [80]:
get_similar('фотосессия', count=5)

,similarity
word,
фотограф,0.815179
фотоаппарат,0.726092
штатив,0.693431
фотопортрет,0.681604
снимки,0.675059


In [81]:
get_similar('научный сотрудник')

,similarity
word,
учёная степень,0.803709
академик,0.801166
доктор наук,0.789567
доцент,0.75874
диссертация,0.739188
аспирант,0.724107
бакалавриат,0.713947
магистр,0.701732
учёный совет,0.69729


In [82]:
def diminutive(word, count=3):
    """Уменьшительно-ласкательная форма"""
    return get_similar(word + ',коробочка,-коробка,пуговка,-пуговица,носик,-нос,пёсик,-пёс,ножка,-нога,речка,-река', count=count)

In [83]:
diminutive('крышка', count=1)

,similarity
word,
крышечка,0.336946


In [84]:
diminutive('медведь', count=1)

,similarity
word,
заинька,0.328291


In [85]:
diminutive('мышь', count=1)

,similarity
word,
мышка,0.420869


In [86]:
diminutive('рана', count=1)

,similarity
word,
ранка,0.339992


In [87]:
diminutive('трусы', count=1)

,similarity
word,
трусики,0.38418


In [88]:
diminutive('заяц', count=1)

,similarity
word,
зайка,0.426464


In [89]:
diminutive('ночь', count=3)

,similarity
word,
сумерки,0.332968
вечер,0.301711
полумрак,0.291707


In [90]:
get_similar('фифа', count=5)

,similarity
word,
корнер,0.59283
пенальти,0.582125
футбол,0.566727
гол,0.561703
арбитр,0.560168


In [91]:
get_similar('фифа,-футбол', count=5)

,similarity
word,
краля,0.411737
красотка,0.357701
цаца,0.346431
фря,0.319825
кокетка,0.30955


In [92]:
get_similar('няша', count=5)

,similarity
word,
няшность,0.731425
кавайность,0.654609
анимешница,0.635502
мимимишечки,0.600719
некоджиру,0.570457


In [93]:
get_similar('инь-ян')

,similarity
word,
дуальность,0.656545
противоположность,0.617736
дао,0.61291
мужчина и женщина,0.598739
дуализм,0.587418
бинарность,0.577384
биполярность,0.562219
инь,0.556854
венера и марс,0.538925


In [94]:
get_similar('мука,-мучения', count=5)

,similarity
word,
хлеб,0.575102
тесто,0.536978
выпечка,0.519722
булка,0.517207
сдоба,0.509264


In [95]:
get_similar('мука,-хлеб', count=5)

,similarity
word,
муки,0.409322
мучение,0.392532
мучения,0.354434
страдание,0.342737
страдания,0.340198


In [96]:
get_similar('своды')

,similarity
word,
свод,0.70674
купола,0.637788
купол,0.627816
арки,0.583827
анфилада,0.572934
колоннада,0.571528
колонны,0.560099
стрельчатость,0.551313
стрельчатые окна,0.518981


In [97]:
get_similar('резон')

,similarity
word,
причина,0.60133
обоснование,0.596741
факторы,0.596654
отговорка,0.563005
довод,0.549096
повод,0.511486
тезис,0.480768
аргумент,0.466554
предпосылка,0.413426


In [98]:
get_similar('брут')

,similarity
word,
изменник,0.538281
предательство,0.497709
предатель,0.473415
братишка,0.46334
друган,0.458825
братан,0.440994
костян,0.429204
бро,0.424638
подлость,0.412965


In [99]:
get_similar('фейсбук')

,similarity
word,
социальная сеть,0.846461
вконтакте,0.794796
соцсети,0.770371
твиттер,0.737744
социальные сети,0.735758
репост,0.691864
блог,0.596908
твит,0.579559
комменты,0.576611


In [100]:
get_similar('воодушевление')

,similarity
word,
энтузиазм,0.540558
преисполненность,0.537265
окрылённость,0.529066
вдохновение,0.512462
энтузиаст,0.471517
восторг,0.444064
упоение,0.403413
оптимизм,0.40116
время вперёд,0.39342


In [101]:
get_similar('наставник')

,similarity
word,
тьютор,0.669204
учитель,0.651381
ментор,0.59822
подмастерье,0.569184
гуру,0.540821
йода,0.516008
педагог,0.514194
коуч,0.50607
репетитор,0.5022


In [102]:
get_similar('кот')

,similarity
word,
кошка,0.80653
котик,0.689909
мурчание,0.624404
котёнок,0.603758
мяуканье,0.602267
мурлыканье,0.584347
котейка,0.568221
киска,0.559012
котофей,0.546554


In [103]:
get_similar('гекльберри финн')

,similarity
word,
том сойер,0.811234
марк твен,0.635456
миссисипи,0.483548
архаровец,0.463221
мальчуган,0.395992
приключения,0.386857
похождения,0.384633
вандал,0.375313
оторванность,0.374847


In [104]:
get_similar('лён', 7)

,similarity
word,
хлопок,0.564454
ткань,0.5427
ситец,0.53012
сатин,0.505154
текстиль,0.485103
шёлк,0.475612
мережка,0.471778


In [105]:
get_similar('глава,-начальник', 5)

,similarity
word,
оглавление,0.521155
параграф,0.513125
заголовок,0.508924
заглавие,0.500733
абзац,0.472904


In [106]:
get_similar('глава,-параграф', 5)

,similarity
word,
главный,0.473021
управленец,0.471149
главарь,0.452533
руководитель,0.437571
начальник,0.436371


In [107]:
get_similar('бревно')

,similarity
word,
брёвна,0.750354
сруб,0.733948
полено,0.712265
древесина,0.691047
лесопилка,0.663169
пилорама,0.650263
лесоруб,0.629687
дрова,0.627147
лесоматериал,0.621442


In [108]:
get_similar('опасность')

,similarity
word,
тревога,0.576818
угроза,0.570873
осторожность,0.544884
опаска,0.533993
опасение,0.527395
предупреждение,0.525193
страх,0.515052
чрезвычайное положение,0.510376
паника,0.510152


In [109]:
get_similar('хенд-мейд')

,similarity
word,
ручная работа,0.681002
крафт,0.668359
рукоделие,0.661784
поделка,0.568757
народный промысел,0.567894
макраме,0.556684
бисероплетение,0.553865
ремесло,0.533489
мезенка,0.532838


In [110]:
get_similar('дженис джоплин')

,similarity
word,
рок-н-ролл,0.520339
блюз,0.509373
джаз,0.502814
шестидесятые,0.498489
джеф линн,0.492721
рокабилли,0.49261
рэй чарльз,0.487358
соул,0.473848
свинг,0.470644


In [111]:
get_similar('свежий воздух')

,similarity
word,
проветривание,0.644819
форточка,0.59864
сквозняк,0.524124
воздух,0.519538
дуновение,0.49225
свежесть,0.485019
вдох,0.482993
ветерок,0.472331
окно,0.453963


In [112]:
get_similar('клоун')

,similarity
word,
шут,0.749671
клоуны,0.714213
комик,0.691403
цирк,0.660056
шутник,0.637759
циркач,0.635689
юморист,0.622481
шапито,0.621861
смех,0.615228


In [113]:
get_similar('дороговизна')

,similarity
word,
цена,0.629422
стоимость,0.540103
ценник,0.531296
цены,0.529545
подорожание,0.528591
себестоимость,0.497815
бутик,0.486948
прайс,0.453561
наценка,0.452635


In [114]:
get_similar('дешёвка')

,similarity
word,
дешевизна,0.562569
стекляшки,0.540175
суррогат,0.532368
подделка,0.529027
абибас,0.523254
эрзац,0.521988
фальшивка,0.498892
стразы,0.496323
поношенность,0.49591


In [115]:
get_similar('предчувствие')

,similarity
word,
интуиция,0.790021
чуйка,0.784226
чутьё,0.757249
наитие,0.750825
шестое чувство,0.727117
предвидение,0.6239
проницательность,0.567968
внутренний голос,0.555941
чуткость,0.537105


In [116]:
get_similar('многоугольник')

,similarity
word,
многогранник,0.86808
тетраэдр,0.860218
икосаэдр,0.851642
октаэдр,0.846791
додекаэдр,0.826632
параллелепипед,0.818058
семигранник,0.804067
куб,0.779876
ромб,0.757902


In [117]:
get_similar('Передача')

,similarity
word,
телевизор,0.676844
телевидение,0.676609
телепередача,0.668002
телеканал,0.636768
вещание,0.622791
тв,0.61852
радиопередача,0.597437
новости,0.580623
диктор,0.570573


In [118]:
get_similar('головная боль', 15)

,similarity
word,
аспирин,0.731803
парацетамол,0.731504
нурофен,0.721425
анальгин,0.718951
анальгетик,0.690356
ибупрофен,0.663911
болеутоляющее,0.650086
обезболивающее,0.6376
мигрень,0.615379


In [119]:
get_similar('Лазурит')

,similarity
word,
хризолит,0.831098
яхонт,0.828483
бирюза,0.816959
яшма,0.802459
топаз,0.771393
сапфир,0.74907
самоцвет,0.746411
аквамарин,0.71046
аметист,0.682344


In [120]:
get_similar('Хардкор')

,similarity
word,
хард,0.588265
блэк сэбэт,0.478515
рейд массив,0.473292
тяжелый рок,0.444848
хеви-метал,0.432122
рок,0.42609
треш,0.425667
тяжёлая музыка,0.417408
рамштайн,0.410385


In [121]:
get_similar('Лаконизм')

,similarity
word,
немногословие,0.815482
лаконичность,0.797907
краткость,0.614941
мудрословие,0.613669
словоблудие,0.563124
риторика,0.558517
красноречие,0.551831
заковыристость,0.515189
демагогия,0.507344


In [122]:
get_similar('шарик,-бобик')

,similarity
word,
шар,0.496183
воздушный шарик,0.421158
шары,0.403534
шарики,0.383984
гелиосфера,0.327495
гелий,0.321775
воздушный шар,0.32049
стратостат,0.314767
аэростат,0.302901


In [123]:
get_similar('шарик,-шар')

,similarity
word,
тобик,0.504369
бобик,0.47858
полкан,0.458073
дружок,0.451224
тузик,0.439573
пёс,0.420548
барбос,0.40892
пёсик,0.408312
барсик,0.401958


In [124]:
get_similar('Антон Макаренко')  # педагог

,similarity
word,
беспризорники,0.693475
монтессори,0.603064
воспитатель,0.566132
педагогика,0.56427
педагог,0.501322
тьютор,0.496234
воспитание,0.463213
нравоучение,0.4585
защита детей,0.454088


In [125]:
get_similar('Пётр нестеров')  # лётчик

,similarity
word,
мёртвая петля,0.760387
светлана капанина,0.750647
пилотаж,0.748504
высший пилотаж,0.735456
покрышкин,0.731463
ас,0.680691
лётчик,0.650568
русские горки,0.643044
авиатор,0.585397


In [126]:
get_similar('Коллективный разум', 6)

,similarity
word,
всемирная паутина,0.566438
ноосфера,0.55297
групповое мышление,0.533442
интранет,0.49996
нейронная сеть,0.487668
блогосфера,0.470984


In [127]:
get_similar('гладь')

,similarity
word,
тишь,0.568164
озеро,0.529812
рябь,0.511082
водомерка,0.506955
штиль,0.502641
поверхность,0.450516
тишина,0.437248
глиссер,0.436092
спокойствие,0.40363


In [128]:
get_similar('тишь')

,similarity
word,
тишина,0.803315
тишок,0.723627
затишье,0.712546
безмолвие,0.592947
гладь,0.568164
спокойствие,0.557825
зори,0.55211
беззвучие,0.538501
молчок,0.51894


In [129]:
compare('гладь', 'тишь', 25)

,гладь,score1,общие,score_common,тишь,score2
0,поверхность,0.450516,тишина,0.592661,глушь,0.447683
1,вышивание,0.367484,тишок,0.531401,безмолвие,0.419027
2,глиссер,0.363023,затишье,0.527047,беззвучие,0.408571
3,гладкость,0.358308,штиль,0.507885,молчок,0.389592
4,отражение,0.346725,спокойствие,0.474505,вечерний звон,0.360123
5,пяльцы,0.337593,озеро,0.455212,шорох,0.345309
6,вышивка,0.335468,рябь,0.451037,глухота,0.338383
7,зеркальность,0.324958,покой,0.441797,молчание,0.327174
8,байкал,0.317753,зори,0.429628,ночь,0.322049
9,лодка,0.313133,безмятежность,0.416102,шёпот,0.320975


In [130]:
get_similar('метафора')

,similarity
word,
иносказание,0.599561
аллегория,0.569914
аллюзия,0.569292
памфлет,0.45792
метаморфоза,0.452209
гипербола,0.389316
изъяснение,0.384458
трансформация,0.38154
аналогия,0.368742


In [131]:
get_similar('Хофман')

,similarity
word,
лизергин,0.606088
психоделический опыт,0.546411
лсд,0.542893
холотропное дыхание,0.520178
кен кизи,0.516563
трип,0.511011
травление,0.501956
психостимулятор,0.494711
ацидоз,0.481643


In [132]:
get_similar('Три-Д,-трёхмерность')

,similarity
word,
задний ряд,0.393594
последний ряд,0.318438
кинофильм,0.308582
кинотеатр,0.307896
спецэффекты,0.305174
кинозал,0.301305
киноварь,0.295527
проектор,0.292637
кинокартина,0.287219


In [133]:
get_similar('Три-Д,-задний ряд')

,similarity
word,
стереометрия,0.488763
трёхмерность,0.452948
тессеракт,0.37411
кубатура,0.362837
кубометр,0.361602
куб,0.304185
число авогадро,0.295761
оттиск,0.292688
гиперкуб,0.284369


In [134]:
get_similar('Почта России')

,similarity
word,
почта,0.842562
бандероль,0.836814
почта ссср,0.836662
почтамт,0.823805
посылка,0.82039
почтальон,0.783207
пересылка,0.78244
конверт,0.758983
адресат,0.756805


In [135]:
get_similar('Рэмбо', 5)

,similarity
word,
чак норрис,0.437867
рокки,0.405426
боевик,0.391441
борец,0.380126
сильвестр сталлоне,0.376598


In [136]:
get_similar('допинг,стимулятор', 7)

,similarity
word,
амфетамин,0.711883
спиды,0.648067
наркотик,0.579649
энергетик,0.565125
героин,0.543503
кофеин,0.539895
кокаин,0.524426


In [137]:
get_similar('Зубчики', 3)

,similarity
word,
зубец,0.702631
зубцы,0.673297
зубья,0.65472


In [138]:
get_similar('Мимолётность')

,similarity
word,
мгновение,0.665831
мгновения,0.603884
миг,0.586122
секунды,0.553919
быстротечность,0.550291
секунда,0.533042
кратковременность,0.492508
минута,0.479099
минуты,0.478646


In [139]:
get_similar('впуск')

,similarity
word,
инжектор,0.61804
жиклёр,0.599527
форсунка,0.575787
клапан,0.549625
впрыск,0.521151
запуск,0.513548
карбюратор,0.504001
двигатель,0.480336
поршень,0.475128


In [140]:
get_similar('елена малышева')

,similarity
word,
диетолог,0.470827
леонард маккой,0.470775
история болезни,0.463486
докторишка,0.458394
врачиха,0.44074
здравие,0.396371
уролог,0.389064
невропатолог,0.38479
врачеватель,0.369757


In [141]:
get_similar('я-центризм', 20)

,similarity
word,
эгоцентризм,0.754426
самомнение,0.738465
эго,0.70205
самолюбие,0.701484
самовлюбленность,0.69302
себялюбие,0.691028
яканье,0.68078
эгоизм,0.673091
эгоист,0.661061


In [143]:
get_similar('обязанности', 20)

,similarity
word,
обязанность,0.821615
обязательство,0.717186
обязательства,0.683181
ответственность,0.606841
долг,0.547854
супружеский долг,0.547373
поручительство,0.546224
исполнительность,0.514642
требование,0.492283
